In [1]:
from openff.qcsubmit.results import OptimizationResultCollection
from openff.qcsubmit.results.filters import ConformerRMSDFilter
from openff.toolkit import Molecule, Topology, ForceField
import copy, itertools

In [2]:
from openff.qcsubmit.results.filters import ResultFilter, SinglepointRecordFilter, SinglepointRecordGroupFilter
from typing import List, Optional, Set, Tuple, TypeVar, Union
from openff.qcsubmit._pydantic import (
    BaseModel,
    Field,
    PrivateAttr,
    root_validator,
    validator,
)
from qcportal.optimization import OptimizationRecord
from qcportal.record_models import BaseRecord, RecordStatusEnum
from qcportal.singlepoint import SinglepointRecord
import numpy

# Identicaly to ConformerRMSDFilter, but with the molecule's atoms being canonically ordered
class ConformerRMSDFilterCanonical(SinglepointRecordGroupFilter):
    """A filter which will retain up to a maximum number of conformers for each unique
    molecule (as determined by an entries InChI key) which are distinct to within a
    specified RMSD tolerance.

    Notes:
        * This filter will only be applied to basic and optimization datasets.
          Torsion drive datasets / entries will be skipped.
        * A greedy selection algorithm is used to select conformers which are most
          distinct in terms of their RMSD values.
    """

    max_conformers: int = Field(
        10,
        description="The maximum number of conformers to retain for each unique molecule.",
    )

    rmsd_tolerance: float = Field(
        0.5,
        description="The minimum RMSD [A] between two conformers for them to be "
        "considered distinct.",
    )
    heavy_atoms_only: bool = Field(
        True,
        description="Whether to only consider heavy atoms when computing the RMSD "
        "between two conformers.",
    )
    check_automorphs: bool = Field(
        True,
        description="Whether to consider automorphs when computing the RMSD between two "
        "conformers. Setting this option to ``True`` may slow down the filter "
        "considerably if ``heavy_atoms_only`` is set to ``False``.",
    )

    def _compute_rmsd_matrix_rd(self, molecule: Molecule) -> numpy.ndarray:
        """Computes the RMSD between all conformers stored on a molecule using an RDKit
        backend."""

        from rdkit import Chem
        from rdkit.Chem import AllChem

        rdkit_molecule: Chem.RWMol = molecule.to_rdkit()

        if self.heavy_atoms_only:
            rdkit_molecule = Chem.RemoveHs(rdkit_molecule)

        n_conformers = len(molecule.conformers)
        conformer_ids = [conf.GetId() for conf in rdkit_molecule.GetConformers()]

        rmsd_matrix = numpy.zeros((n_conformers, n_conformers))

        for i, j in itertools.combinations(conformer_ids, 2):
            if self.check_automorphs:
                rmsd_matrix[i, j] = AllChem.GetBestRMS(
                    rdkit_molecule,
                    rdkit_molecule,
                    conformer_ids[i],
                    conformer_ids[j],
                )

            else:
                rmsd_matrix[i, j] = AllChem.GetConformerRMS(
                    rdkit_molecule,
                    conformer_ids[i],
                    conformer_ids[j],
                )

        rmsd_matrix += rmsd_matrix.T
        return rmsd_matrix

    def _compute_rmsd_matrix_oe(self, molecule: Molecule) -> numpy.ndarray:
        """Computes the RMSD between all conformers stored on a molecule using an OpenEye
        backend."""

        from openeye import oechem

        oe_molecule: oechem.OEMol = molecule.to_openeye()
        oe_conformers = {
            i: oe_conformer for i, oe_conformer in enumerate(oe_molecule.GetConfs())
        }

        n_conformers = len(molecule.conformers)

        rmsd_matrix = numpy.zeros((n_conformers, n_conformers))

        for i, j in itertools.combinations([*oe_conformers], 2):
            rmsd_matrix[i, j] = oechem.OERMSD(
                oe_conformers[i],
                oe_conformers[j],
                self.check_automorphs,
                self.heavy_atoms_only,
                True,
            )

        rmsd_matrix += rmsd_matrix.T
        return rmsd_matrix

    def _compute_rmsd_matrix(self, molecule: Molecule) -> numpy.ndarray:
        """Computes the RMSD between all conformers stored on a molecule."""

        try:
            rmsd_matrix = self._compute_rmsd_matrix_rd(molecule)
        except ModuleNotFoundError:
            rmsd_matrix = self._compute_rmsd_matrix_oe(molecule)

        return rmsd_matrix

    def _filter_function(
        self,
        entries: List[
            Tuple[
                "_BaseResult",
                Union[SinglepointRecord, OptimizationRecord],
                Molecule,
                str,
            ]
        ],
    ) -> List[Tuple["_BaseResult", str]]:
        # Sanity check that all molecules look as we expect.
        assert all(molecule.n_conformers == 1 for _, _, molecule, _ in entries)

        # Condense the conformers into a single molecule.
        conformers = [
            molecule.canonical_order_atoms().conformers[0]
            for _, _, molecule, _ in entries
        ]

        [_, _, molecule, _] = entries[0]

        molecule = copy.deepcopy(molecule.canonical_order_atoms())
        molecule._conformers = conformers

        rmsd_matrix = self._compute_rmsd_matrix(molecule)
        # print(rmsd_matrix)

        # Select a set N maximally diverse conformers which are distinct in terms
        # of the RMSD tolerance.

        # Apply the greedy selection process.
        closed_list = numpy.zeros(self.max_conformers).astype(int)
        closed_mask = numpy.zeros(rmsd_matrix.shape[0], dtype=bool)

        n_selected = 1

        for i in range(min(molecule.n_conformers, self.max_conformers - 1)):
            distances = rmsd_matrix[closed_list[: i + 1], :].sum(axis=0)

            # Exclude already selected conformers or conformers which are too similar
            # to those already selected.
            closed_mask[
                numpy.any(
                    rmsd_matrix[closed_list[: i + 1], :] < self.rmsd_tolerance, axis=0
                )
            ] = True

            if numpy.all(closed_mask):
                # Stop of there are no more distinct conformers to select from.
                break

            distant_index = numpy.ma.array(distances, mask=closed_mask).argmax()
            closed_list[i + 1] = distant_index

            n_selected += 1

        return [
            (entries[i.item()][0], entries[i.item()][-1])
            for i in closed_list[:n_selected]
        ]

In [3]:
import numpy as np

# This is just yoinked from ConformerRMSDFilter
def rmsd_rdkit(molecule,check_automorphs,heavy_atoms_only):

    from rdkit import Chem
    from rdkit.Chem import AllChem

    rdkit_molecule: Chem.RWMol = molecule.to_rdkit()

    if heavy_atoms_only:
        rdkit_molecule = Chem.RemoveHs(rdkit_molecule)

    n_conformers = len(molecule.conformers)
    conformer_ids = [conf.GetId() for conf in rdkit_molecule.GetConformers()]

    rmsd_matrix = np.zeros((n_conformers, n_conformers))

    for i, j in itertools.combinations(conformer_ids, 2):
        if check_automorphs:
            rmsd_matrix[i, j] = AllChem.GetBestRMS(
                rdkit_molecule,
                rdkit_molecule,
                conformer_ids[i],
                conformer_ids[j],
            )

        else:
            rmsd_matrix[i, j] = AllChem.GetConformerRMS(
                rdkit_molecule,
                conformer_ids[i],
                conformer_ids[j],
            )

    rmsd_matrix += rmsd_matrix.T
    return rmsd_matrix

# Training dataset

In [6]:
# Using new environment
sage_220_training = OptimizationResultCollection.parse_file('./02_curate-data/output/optimization-training-set-nocanon_newqcp.json')

In [5]:
new_conf_filter_training = OptimizationResultCollection.parse_file('./02_curate-data/output/optimization-training-set.json')

In [7]:
print(sage_220_training.n_molecules,sage_220_training.n_results)
print(new_conf_filter_training.n_molecules,new_conf_filter_training.n_results)

1607 5119
1607 5114


In [8]:
qcaids_newfilter = [entry.record_id for entry in new_conf_filter_training.entries['https://api.qcarchive.molssi.org:443/']]

In [9]:
qcaids_oldfilter = [entry.record_id for entry in sage_220_training.entries['https://api.qcarchive.molssi.org:443/']]

In [10]:
qcaids_both = np.isin(qcaids_oldfilter,qcaids_newfilter)
np.count_nonzero(qcaids_both)

5113

In [11]:
qcaids_oldonly = np.logical_not(np.isin(qcaids_oldfilter,qcaids_newfilter))
np.count_nonzero(qcaids_oldonly)

6

In [12]:
qcaids_newonly = np.logical_not(np.isin(qcaids_newfilter,qcaids_oldfilter))
np.count_nonzero(qcaids_newonly)

1

6 conformers were removed with the new filter, and 1 new ones were added. This should reflect both conformers that were erroneously kept due to the atom mis-ordering, as well as conformers that were not correctly selected to represent the most diverse options.

## Look at the filtered molecules

### Comparing the number of conformers that appear only with the old filter, vs only with the new filter.

In [13]:
qcaids_oldonly_ids = np.array(qcaids_oldfilter)[qcaids_oldonly]
qcaids_newonly_ids = np.array(qcaids_newfilter)[qcaids_newonly]

In [14]:
entries_oldonly = [entry for entry in sage_220_training.entries['https://api.qcarchive.molssi.org:443/'] if entry.record_id in qcaids_oldonly_ids]
ds_oldonly = OptimizationResultCollection.parse_obj({'entries':{'https://api.qcarchive.molssi.org:443/':entries_oldonly}})
records_oldonly = ds_oldonly.to_records()

In [15]:
entries_newonly = [entry for entry in new_conf_filter_training.entries['https://api.qcarchive.molssi.org:443/'] if entry.record_id in qcaids_newonly_ids]
ds_newonly = OptimizationResultCollection.parse_obj({'entries':{'https://api.qcarchive.molssi.org:443/':entries_newonly}})
records_newonly = ds_newonly.to_records()

In [16]:
inchis_newonly = [entry.inchi_key for entry in entries_newonly]
inchis_oldonly = [entry.inchi_key for entry in entries_oldonly]

In [17]:
oldonly_count = {i:inchis_oldonly.count(i) for i in inchis_oldonly}
newonly_count = {i:inchis_newonly.count(i) for i in inchis_newonly}

In [18]:
not_one_to_one_new = []
print("InChI Key         ","N_new_only","N_old_only")
for key in newonly_count.keys():
    try:
        print(key,newonly_count[key],oldonly_count[key])
        if newonly_count[key] != oldonly_count[key]:
            not_one_to_one_new.append(key)
    except KeyError:
        print(key,newonly_count[key])
        not_one_to_one_new.append(key)

print(not_one_to_one_new)

InChI Key          N_new_only N_old_only
BMAMZZAGBSHIHM-XGRDREAVNA-O 1 1
[]


Here we look at conformers that appear only in the new filter's results. The only result here is one where one conformer was eliminated and one new one was added to replace it. This is probably due to the diversity metric.

In [20]:
not_one_to_one_old = []
print("InChI Key         ","N_new_only","N_old_only")
for key in oldonly_count.keys():
    try:
        print(key,newonly_count[key],oldonly_count[key])
        if newonly_count[key] != oldonly_count[key]:
            not_one_to_one_old.append(key)
    except KeyError:
        print(key,' ',oldonly_count[key])
        not_one_to_one_old.append(key)

print(not_one_to_one_old)

InChI Key          N_new_only N_old_only
RORAOHDOEDEVOG-YHMJCDSINA-N   1
IWKUWQHVHPGHMC-JDBSGULFNA-N   2
DTBNBXWJWCWCIK-HOVPBEAZNA-K   1
WUAGPRSQTJSTRA-QVNRZVRXNA-M   1
BMAMZZAGBSHIHM-XGRDREAVNA-O 1 1
['RORAOHDOEDEVOG-YHMJCDSINA-N', 'IWKUWQHVHPGHMC-JDBSGULFNA-N', 'DTBNBXWJWCWCIK-HOVPBEAZNA-K', 'WUAGPRSQTJSTRA-QVNRZVRXNA-M']


Most of the conformers that only appear in the old dataset were not replaced by anything in the new dataset, suggesting they were filtered out for being too similar to other conformers.

### Compare the actual conformers that were eliminated
Now we are moving to looking at all conformers for a given molecule that was affected by the filter. 

In [21]:
all_entries_oldonly = [entry for entry in sage_220_training.entries['https://api.qcarchive.molssi.org:443/'] if entry.inchi_key in oldonly_count.keys()]
all_ds_oldonly = OptimizationResultCollection.parse_obj({'entries':{'https://api.qcarchive.molssi.org:443/':all_entries_oldonly}})
all_records_oldonly = all_ds_oldonly.to_records()

In [22]:
all_entries_newonly = [entry for entry in new_conf_filter_training.entries['https://api.qcarchive.molssi.org:443/'] if entry.inchi_key in newonly_count.keys()]
all_ds_newonly = OptimizationResultCollection.parse_obj({'entries':{'https://api.qcarchive.molssi.org:443/':all_entries_newonly}})
all_records_newonly = all_ds_newonly.to_records()

In [23]:
all_entries_oldonly_newds = [entry for entry in new_conf_filter_training.entries['https://api.qcarchive.molssi.org:443/'] if entry.inchi_key in oldonly_count.keys()]
all_ds_oldonly_newds = OptimizationResultCollection.parse_obj({'entries':{'https://api.qcarchive.molssi.org:443/':all_entries_oldonly_newds}})
all_records_oldonly_newds = all_ds_oldonly_newds.to_records()

In [24]:
all_entries_newonly_oldds = [entry for entry in sage_220_training.entries['https://api.qcarchive.molssi.org:443/'] if entry.inchi_key in newonly_count.keys()]
all_ds_newonly_oldds = OptimizationResultCollection.parse_obj({'entries':{'https://api.qcarchive.molssi.org:443/':all_entries_newonly_oldds}})
all_records_newonly_oldds = all_ds_newonly_oldds.to_records()

In [25]:
def group_by_smiles(records,ds):
    mol_by_smiles = {}
    for record in records:
        recordid = record[0].id
        ds_entry = [entry for entry in ds.entries['https://api.qcarchive.molssi.org:443/'] if entry.record_id == recordid]
        if len(ds_entry) > 1:
            print(recordid)
        inchi_key = ds_entry[0].inchi_key
        # smiles = record[1].to_smiles()
        if inchi_key in mol_by_smiles:
            mol_by_smiles[inchi_key].add_conformer(record[1].canonical_order_atoms().conformers[0])
        else:
            mol_by_smiles[inchi_key] = copy.deepcopy(record[1].canonical_order_atoms())
    return mol_by_smiles

In [26]:
oldonly_mols = group_by_smiles(all_records_oldonly,sage_220_training)
newonly_mols = group_by_smiles(all_records_newonly,new_conf_filter_training)

In [27]:
oldonly_mols_newds = group_by_smiles(all_records_oldonly_newds,new_conf_filter_training)
newonly_mols_oldds = group_by_smiles(all_records_newonly_oldds,sage_220_training)

In [28]:
len(oldonly_mols.keys())

5

In [29]:
len(newonly_mols.keys())

1

In [30]:
for key in oldonly_mols.keys():
    print(key,oldonly_mols[key].n_conformers)
    print('RMSD using old filter:\n',rmsd_rdkit(oldonly_mols[key],True,True))
    try:
        print('RMSD using new filter:\n',rmsd_rdkit(oldonly_mols_newds[key],True,True))
        if rmsd_rdkit(oldonly_mols[key],True,True).shape == rmsd_rdkit(oldonly_mols_newds[key],True,True).shape and oldonly_mols[key].n_conformers>1:
            print('Difference in RMSD:\n',rmsd_rdkit(oldonly_mols[key],True,True) - rmsd_rdkit(oldonly_mols_newds[key],True,True))
        elif oldonly_mols[key].n_conformers == 1 and oldonly_mols[key].n_conformers == oldonly_mols_newds[key].n_conformers:
            print('RMSD between old and new conformer:')
            new_mol = copy.deepcopy(oldonly_mols[key])
            new_mol._conformers += oldonly_mols_newds[key]._conformers
            print(rmsd_rdkit(new_mol,True,True)[1,0])
    
    except KeyError:
        print("Not in new dataset")
    print()

RORAOHDOEDEVOG-YHMJCDSINA-N 3
RMSD using old filter:
 [[0.         0.5594225  0.60866356]
 [0.5594225  0.         0.11116209]
 [0.60866356 0.11116209 0.        ]]
RMSD using new filter:
 [[0.         0.60866356]
 [0.60866356 0.        ]]

IWKUWQHVHPGHMC-JDBSGULFNA-N 4
RMSD using old filter:
 [[0.         2.0359773  0.04711509 2.00589999]
 [2.0359773  0.         2.06076145 0.05154587]
 [0.04711509 2.06076145 0.         2.03106422]
 [2.00589999 0.05154587 2.03106422 0.        ]]
RMSD using new filter:
 [[0.        2.0359773]
 [2.0359773 0.       ]]

DTBNBXWJWCWCIK-HOVPBEAZNA-K 2
RMSD using old filter:
 [[0.         0.00026277]
 [0.00026277 0.        ]]
RMSD using new filter:
 [[0.]]

WUAGPRSQTJSTRA-QVNRZVRXNA-M 2
RMSD using old filter:
 [[0.         0.00015141]
 [0.00015141 0.        ]]
RMSD using new filter:
 [[0.]]

BMAMZZAGBSHIHM-XGRDREAVNA-O 6
RMSD using old filter:
 [[0.         1.58266772 1.45507043 0.86227238 1.32501792 0.89907329]
 [1.58266772 0.         0.90792318 1.46781972 0.5

4 molecules had conformers eliminated due to low RMSD (5 total conformers, most with 1, one with 2); not replaced with a new conformer.

1 molecule has the same number of conformers but different structures included due to diversity considerations (1 conformer swapped); the difference between the old and new conformer is small (RMSD diff < 1e-3 A).

In [32]:
for key in newonly_mols.keys():
    if key not in oldonly_mols.keys():
        print(key)
        print(rmsd_rdkit(newonly_mols[key],True,True))
        try:
            print(rmsd_rdkit(newonly_mols_oldds[key],True,True))
        except KeyError:
            print("Not in new dataset")
        if key not in oldonly_mols.keys():
            print('Not in old only keys')
        print()

All molecules with conformers that only appear in the new dataset were also in the old dataset, just with different conformers, so were addressed in the previous cell

In [33]:
print(np.count_nonzero(np.isin(list(oldonly_mols.keys()),list(newonly_mols.keys()))),' keys in common out of ',len(list(oldonly_mols.keys())))
print(np.count_nonzero(np.isin(list(newonly_mols.keys()),list(oldonly_mols.keys()))),' keys in common out of ',len(list(newonly_mols.keys())))

1  keys in common out of  5
1  keys in common out of  1


# Benchmark dataset

In [5]:
sage_220_benchmark = OptimizationResultCollection.parse_file('../sage-2.2.0/05_benchmark_forcefield/datasets/OpenFF-Industry-Benchmark-Season-1-v1.1-filtered-charge-coverage.json')

In [4]:
benchmark_filter = OptimizationResultCollection.parse_file('./05_benchmark_forcefield/datasets/OpenFF-Industry-Benchmark-Season-1-v1.1-ConfRMSD-0p4.json')

In [6]:
print(sage_220_benchmark.n_molecules,sage_220_benchmark.n_results)

9822 76351


In [7]:
print(benchmark_filter.n_molecules,benchmark_filter.n_results)

9822 66978


About 12,000 conformers eliminated

In [8]:
qcaids_filter_bm = [entry.record_id for entry in benchmark_filter.entries['https://api.qcarchive.molssi.org:443/']]

In [9]:
qcaids_nofilter_bm = [entry.record_id for entry in sage_220_benchmark.entries['https://api.qcarchive.molssi.org:443/']]

In [10]:
qcaids_not_filtered = np.isin(qcaids_nofilter_bm,qcaids_filter_bm)
np.count_nonzero(qcaids_not_filtered)

66978

In [11]:
qcaids_filtered_out = np.logical_not(qcaids_not_filtered)
np.count_nonzero(qcaids_filtered_out)

9373

In [12]:
qcaids_fo_ids = np.array(qcaids_nofilter_bm)[qcaids_filtered_out]

In [13]:
inchi_keys_fo = np.unique([entry.inchi_key for entry in sage_220_benchmark.entries['https://api.qcarchive.molssi.org:443/'] if entry.record_id in qcaids_fo_ids])

In [14]:
len(inchi_keys_fo)

4212

almost 5000 affected molecules, out of 10000